# Consume messages from an Aiven for Apache Kafka®️ partitioned topic

In this notebook you will learn how to consume messages to an Apache Kafka topic with two partitions (created with the [dedicated notebook](3-produce-partitioned-topic.ipynb))

![Consume messages from an Apache Kafka Topic](../img/4%20-%20multiple%20kafka%20consumers%20one%20consumer%20group%20python.png)

> [!NOTE]
>
> _If you haven't already created and setup your virtual environment, follow the steps from [0-setup.ipynb](0-setup.ipynb)_. 
>
> You will need the `PizzasPartitioned` topic that you created in `3-produce-partintioned-topic.ipynb`.



---

## Create an Apache Kafka®️ Consumer

The next step is to configure the Kafka Consumer. let's recall in the previous notebook. That `Francesco's` message was sent to partition `0` and `Adele's` message was in partition `1`.

```python
producer.produce(
    "pizzaPartitioned",
    key={"id":1},
    value={"id":1, "name":"👨 Francesco", "pizza":"Margherita 🍕"},
    partition=0
)

producer.produce(
    "pizzaPartitioned",
    key={"id":2},
    value={"id":2, "name":"👩 Adele", "pizza":"Hawaii 🍕+🍍+🥓"},
    partition=1
)
```

In `2-consume.ipynb` we saw that all the topics are all loaded. We're going to create two consumers that belong to the group `pizzaaioli`. `consumer_0` will consume messages in partition `0` and `consumer_1` consumes messages in partition `1`.

In [1]:
import json
import os

from confluent_kafka import DeserializingConsumer, TopicPartition
from dotenv import load_dotenv

load_dotenv()


def json_serializer(msg, s_obj):
    return json.loads(msg.decode('ascii'))

KAFKA_SERVICE_URI = os.getenv("KAFKA_SERVICE_URI")
topic = "pizzaPartitioned"
conf = {
    'bootstrap.servers': KAFKA_SERVICE_URI,
    'client.id': 'myclient',
    'group.id': 'pizzaioli',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key', 
    'value.deserializer': json_serializer,
    'key.deserializer': json_serializer
    }

orders_consumer_store_0 = DeserializingConsumer(conf)
orders_consumer_store_0.assign([TopicPartition(topic, 0)])

orders_consumer_store_1 = DeserializingConsumer(conf)
orders_consumer_store_1.assign([TopicPartition(topic, 1)])

## Consume messages from the `pizzaPartitioned` topic

Review and start the code block below. While the block is running. Use the **LAST** block in `3-produce-partitioned-topic.ipynb` to produced more partitioned samples.

In [2]:
from confluent_kafka import TopicPartition, KafkaException
from rich.console import Console

console = Console()
def display_orders(consumer:DeserializingConsumer, label:str, style:str = "bold", count:int = 1):
    """Display the orders from a consumer."""
    finished = False
    local_count = 0
    while not finished:
        if (msg:=consumer.poll(timeout=1.0)) is None:
            continue
        elif msg.error():
                raise KafkaException(msg.error())
        else:
            console.print(f"Checking Partition {label} \n" + "-"*10, style=style)
            console.print(f"[{style}]{msg.partition()}:{msg.offset()}: {msg.key()}:{msg.value()}[/{style}]\n\n")
            local_count += 1
            finished = local_count == count

console.print("Run the last block from `Produce-Partitioned-Topics.ipynb`.")
display_orders(orders_consumer_store_0, "Northern Store", "bold red")
display_orders(orders_consumer_store_1, "Southern Store", "bold green")

Run the last block from `Produce-Partitioned-Topics.ipynb`.

Checking Partition Northern Store 
----------

0:35: {'id': 1}:{'id': 1, 'name': '👨 Francesco', 'pizza': 'Margherita 🍕'}

Checking Partition Southern Store 
----------

1:25: {'id': 2}:{'id': 2, 'name': '👩 Adele', 'pizza': 'Hawaii 🍕+🍍+🥓'}

%5|1710520605.533|REQTMOUT|myclient#consumer-1| [thrd:ssl://34.118.134.124:14181/2]: ssl://34.118.134.124:14181/2: Timed out FetchRequest in flight (after 913917ms, timeout #0)
%4|1710520605.533|REQTMOUT|myclient#consumer-1| [thrd:ssl://34.118.134.124:14181/2]: ssl://34.118.134.124:14181/2: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%5|1710520605.533|REQTMOUT|myclient#consumer-2| [thrd:ssl://34.95.2.170:14181/1]: ssl://34.95.2.170:14181/1: Timed out FetchRequest in flight (after 913911ms, timeout #0)
%4|1710520605.533|REQTMOUT|myclient#consumer-2| [thrd:ssl://34.95.2.170:14181/1]: ssl://34.95.2.170:14181/1: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1710520605.533|FAIL|myclient#consumer-2| [thrd:ssl://34.95.2.170:14181/1]: ssl://34.95.2.170:14181/1: 1 request(s) timed out: disconnect (after 3264771ms in state UP)
%3|1710520605.533|FAIL|myclient#consumer-1| [thrd:ssl://34.118.134.124:14181/2]: ssl://34.118.134.124:

### Excellent Work! 🥳

In the next notebook we'll expand on this concept by creating multiple consumer groups so that we can adjust how partitions are consumed. [Click here to get started](5-new-consumer-group.ipynb)